In [1]:
cd ..

/home/jovyan


In [2]:
%run lib/helper/load_rec_mats.py

In [4]:
del item_bias

In [5]:
del exp_mat

In [6]:
import numpy as np

In [9]:
np.min(item_mat)

-0.71530697023799727

In [10]:
np.min(feat_mat)

-0.34598041926981099

In [3]:
%run lib/helper/load_search.py

In [6]:
from lib.recommend import get_sim
import numpy as np

In [5]:
class Recommender():
    
    def __init__(self, n_games=10000):
        self.u_vec = np.zeros(n_games)
        self.feat_bag = set()
        self.added_to_bag = set()
        self.liked = set()
        self.rated = set()
        self.item_bag = set()
        self.last_get = 0
        
    def rate(self, gs):
        self.u_vec[gs] = 1
        self.liked.add(gs)
        self.rated.add(gs)
        
    def get_bag(self, K=100):
        to_get = self.liked.difference(self.added_to_bag)
        while to_get:
            game = to_get.pop()
            top_feat_items, sim = get_sim.by_feats(game, K, feat_mat)
            self.feat_bag.update(top_feat_items)
            top_item_items, c_sim = get_sim.by_items(game, K, item_mat)
            self.item_bag.update(top_item_items)
            self.added_to_bag.add(game)
        
    def clean_bag(self):
        self.feat_bag.difference_update(self.rated)
        self.item_bag.difference_update(self.rated)
        exps = set(exp_mat[list(self.rated)].nonzero()[1])
        self.feat_bag.difference_update(exps)
        self.item_bag.difference_update(exps)
        
    def update_bag_vec(self):
        self.feat_bag_vec = np.zeros_like(self.u_vec)
        self.feat_bag_vec[list(self.feat_bag)] = 1
        self.item_bag_vec = np.zeros_like(self.u_vec)
        self.item_bag_vec[list(self.item_bag)] = 1
        
    def get_cum_vecs(self, item_weight=.7, feat_weight=.3):
        rated_sim_bag = np.multiply(item_mat[list(self.rated)], self.feat_bag_vec)
        cum_corr_vec = np.sum(rated_sim_bag, axis=0)
        self.bias_corr = np.multiply(cum_corr_vec, item_bias)
        rated_corr_bag = np.multiply(feat_mat[list(self.rated)], self.item_bag_vec)
        cum_sim_vec = np.sum(rated_corr_bag, axis=0)
        self.bias_sim = np.multiply(cum_sim_vec, item_bias)
        self.comb_vec = np.add(np.multiply(self.bias_corr, item_weight), np.multiply(self.bias_sim, feat_weight))
        
    def get(self, n):
        if len(self.rated) > self.last_get:
            self.get_bag()
            self.clean_bag()
            self.update_bag_vec()
            self.get_cum_vecs()
        comb_recs = np.argsort(self.comb_vec)[:-(n+1):-1]
        self.top_games = search_gname(comb_recs)
        _ = [print(game) for game in self.top_games]
        self.last_get = len(self.rated)

In [ ]:
def search_gname(gs_list):
    '''
    Return of list of gnames from list of gs.
    '''
    return [gname[gs] for gs in gs_list]

In [7]:
global feat_mat
global item_mat
global exp_mat
global gname

In [20]:
New_user = Recommender()

In [22]:
New_user.rate(gs['King of Tokyo'])

In [23]:
New_user.rate(gs['Agricola'])

In [24]:
New_user.rate(gs['Magic: The Gathering'])

In [25]:
New_user.get(50)

Masmorra de DADOS
In Flanders Field
Le Havre
Fields of Arle
Nusfjord
Pixel Tactics 4
Ninja Dice: Location Cards
Tomb Raider Collectible Card Game
Capo Dei Capi
Tiny Epic Galaxies Deluxe Edition
Food Chain
Summoner Wars: Phoenix Elves vs Tundra Orcs
Summoner Wars: Guild Dwarves vs Cave Goblins
Force of Will
Warhammer: Invasion
Risk Legacy
Star Wars: Destiny – Rey Starter Set
The Voyages of Marco Polo
Summoner Wars: Master Set
World of Warcraft Trading Card Game
Nations
Last Will
Belfort
Agricola: All Creatures Big and Small – More Buildings Big and Small
Pixel Tactics 2
Lewis & Clark
La Granja
Lords of War:  Elves versus Lizardmen
Machi Koro: Deluxe Edition
Suburbia
Cuba: El Presidente
Alien Frontiers
Dojo Kun
Viticulture: Arboriculture Expansion
Risk: Game of Thrones
Terraforming Mars
Las Vegas
Star Wars: Destiny – Kylo Ren Starter Set
Ashes: Rise of the Phoenixborn
Yamslam
Steam Park
At the Gates of Loyang
Mombasa
Viticulture Essential Edition
Tournay
Pixel Tactics 3
Würfel Bohnanza
H